In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
exp_phase_kinetics.py
~~~~~~~~~~~~~~~~~~~~~
Two-point kinetic and stoichiometric calculations for the **exponential
phase** of CHO fed-batch cultures (Clone × Rep).

Workflow
--------
1. Ask the user for the exponential-phase window (`t_start`, `t_end` in h).
2. Load the cleaned experimental CSV (**data/data.csv**) generated by
   FlowJo or your upstream pipeline (metadata row is skipped).
3. Keep only rows within the selected time window.
4. For each *Clone × Rep*:
   • Select the **first** (t₀) and **last** (t₁) samples in the window  
   • Compute specific growth rate µ from VCD at t₀ & t₁  
   • Calculate IVCD as the trapezoid between the two VCD points  
   • Perform total balances for cells, glucose, lactate, glutamine,
     glutamate and recombinant protein (rP)  
   • Derive biomass yields and specific rates  
5. Export:
   • Per-replicate results (`kinetics_by_clone_rep.csv`)  
   • Clone-level summary (mean ± SD, `kinetics_by_clone.csv`).

Inputs
------
data/data.csv  
Expected columns (case-sensitive):  
Clone, Rep, t_hr, VCD, Vol_mL, Glc_g_L, Lac_g_L, Gln_mM, Glu_mM, rP_mg_L

Outputs
-------
• outputs/kinetics_by_clone_rep.csv  – one row per Clone × Rep  
• outputs/kinetics_by_clone.csv      – mean ± SD per Clone

New columns generated
---------------------
mu, IVCD, dX, dG, dL, dQln, dQlu, dP, Y_XG, Y_XL, Y_XQln, Y_XQlu,
q_G, q_L, q_Gln, q_Glu, q_rP

Author
------
Emiliano Balderas R. | 16 Jul 2025  
Last edit : 06 Aug 2025
"""

from pathlib import Path
import numpy as np
import pandas as pd

# ───── Configuration ─────────────────────────────────────────────────── #
DATA_FILE   = Path("data/data.csv")
OUT_REP     = Path("outputs/kinetics_by_clone_rep.csv")
OUT_CLONE   = Path("outputs/kinetics_by_clone.csv")

MM_GLC  = 180.156  # g·mol⁻¹
MM_LAC  =  90.080  # g·mol⁻¹
PG_PER_G = 1e12    # g → pg conversion

# ───── Ask user for time window (if run as script) ───────────────────── #
def get_phase_window():
    try:
        t0 = float(input("Start of exponential phase (h): "))
        t1 = float(input("End   of exponential phase (h): "))
        if t0 >= t1:
            raise ValueError
    except Exception:
        print("⚠️  Invalid input → using default 0–96 h")
        t0, t1 = 0, 96
    return t0, t1

if __name__ == "__main__":
    EXP_START, EXP_END = get_phase_window()
else:
    EXP_START, EXP_END = 0, 96

# ───── Load data ─────────────────────────────────────────────────────── #
if not DATA_FILE.exists():
    raise FileNotFoundError(f"Input file not found: {DATA_FILE}")

raw = pd.read_csv(DATA_FILE, skiprows=1)

# Ensure numeric & tidy categories
num_cols = ["Rep", "t_hr", "Vol_mL",
            "Glc_g_L", "Lac_g_L",
            "Gln_mM", "Glu_mM", "rP_mg_L"]
for col in num_cols:
    raw[col] = pd.to_numeric(raw.get(col), errors="coerce")

df = (
    raw
    .dropna(subset=["Clone", "Rep", "t_hr", "VCD"])
    .assign(Clone=lambda d: d["Clone"].astype("category"),
            Rep=lambda d: d["Rep"].astype("Int64"))
    .query(f"{EXP_START} <= t_hr <= {EXP_END}")
    .sort_values(["Clone", "Rep", "t_hr"], ignore_index=True)
)

# ───── Unit conversions (per-mL basis) ──────────────────────────────── #
df["Glc_mol_mL"] = df["Glc_g_L"] / MM_GLC * 1e3 * 1e-6
df["Lac_mol_mL"] = df["Lac_g_L"] / MM_LAC * 1e3 * 1e-6
df["Gln_mol_mL"] = df["Gln_mM"] * 1e-6
df["Glu_mol_mL"] = df["Glu_mM"] * 1e-6
df["rP_g_mL"]    = df["rP_mg_L"] * 1e-6          # mg·L⁻¹ → g·mL⁻¹

# ───── Two-point kinetics (first vs last row in window) ─────────────── #
results = []

for (clone, rep), g in df.groupby(["Clone", "Rep"], observed=True, sort=False):
    if g.shape[0] < 2:
        continue

    t0, t1 = g.iloc[0], g.iloc[-1]

    Δt = t1["t_hr"] - t0["t_hr"]
    if Δt <= 0:
        continue

    # Growth & IVCD
    mu = (np.log(t1["VCD"]) - np.log(t0["VCD"])) / Δt
    ivcd = ((t0["VCD"] + t1["VCD"]) / 2) * Δt

    # Balances
    dX   = t1["VCD"]*t1["Vol_mL"] - t0["VCD"]*t0["Vol_mL"]
    dG   = t0["Glc_mol_mL"]*t0["Vol_mL"] - t1["Glc_mol_mL"]*t1["Vol_mL"]
    dL   = t1["Lac_mol_mL"]*t1["Vol_mL"] - t0["Lac_mol_mL"]*t0["Vol_mL"]
    dQln = t0["Gln_mol_mL"]*t0["Vol_mL"] - t1["Gln_mol_mL"]*t1["Vol_mL"]
    dQlu = t1["Glu_mol_mL"]*t1["Vol_mL"] - t0["Glu_mol_mL"]*t0["Vol_mL"]
    dP   = t1["rP_g_mL"] *t1["Vol_mL"] - t0["rP_g_mL"] *t0["Vol_mL"]  # g

    # Yields
    Y_XG   = dX/dG   if dG   else np.nan
    Y_XL   = dX/dL   if dL   else np.nan
    Y_XQln = dX/dQln if dQln else np.nan
    Y_XQlu = dX/dQlu if dQlu else np.nan

    # Specific rates (pmol or pg cell⁻¹ h⁻¹)
    q_G   = (dG  *1e12)/ivcd if ivcd else np.nan
    q_L   = (dL  *1e12)/ivcd if ivcd else np.nan
    q_Gln = (dQln*1e12)/ivcd if ivcd else np.nan
    q_Glu = (dQlu*1e12)/ivcd if ivcd else np.nan
    q_rP  = (dP  *PG_PER_G)/ivcd if ivcd else np.nan

    results.append({
        "Clone": clone, "Rep": rep,
        "mu": mu, "IVCD": ivcd,
        "dX": dX, "dG": dG, "dL": dL, "dQln": dQln, "dQlu": dQlu, "dP": dP,
        "Y_XG": Y_XG, "Y_XL": Y_XL, "Y_XQln": Y_XQln, "Y_XQlu": Y_XQlu,
        "q_G": q_G, "q_L": q_L, "q_Gln": q_Gln, "q_Glu": q_Glu, "q_rP": q_rP
    })

kin_df = pd.DataFrame(results)

# ───── Save outputs ─────────────────────────────────────────────────── #
OUT_REP.parent.mkdir(parents=True, exist_ok=True)
kin_df.to_csv(OUT_REP, index=False)
print(f"✓ Saved Clone × Rep kinetics → {OUT_REP}")

agg = (
    kin_df.groupby("Clone", observed=True)
          .agg(["mean", "std"])
          .reset_index()
)
agg.columns = ["Clone"] + [f"{c}_{s}" for c, s in agg.columns[1:]]
agg.to_csv(OUT_CLONE, index=False)
print(f"✓ Saved Clone-level summary  → {OUT_CLONE}")

# ───── CLI summary ──────────────────────────────────────────────────── #
if __name__ == "__main__":
    print("\n=== Exponential-phase kinetics ===")
    print(f"Window : {EXP_START} – {EXP_END} h")
    print(f"Clones : {kin_df['Clone'].nunique()}")
    print(f"Rows   : {kin_df.shape[0]}")


✓ Saved Clone × Rep kinetics → outputs\kinetics_by_clone_rep.csv
✓ Saved Clone-level summary  → outputs\kinetics_by_clone.csv

=== Exponential-phase kinetics ===
Window : 24.0 – 96.0 h
Clones : 3
Rows   : 9


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
plot_exp.py
~~~~~~~~~~~
Bar-chart visualisation of **Clone-level kinetic metrics** calculated for the
exponential phase of CHO fed-batch cultures.

Workflow
--------
1. Load the clone summary produced by `exp_phase_kinetics.py`
   (`outputs/kinetics_by_clone.csv`).
2. For each predefined metric:
   • Read the mean ( *_mean* ) and standard deviation ( *_std* ) columns.  
   • Draw a bar plot (mean ± SD) coloured by Clone.  
3. Save each figure as a high-resolution PNG.

Inputs
------
outputs/kinetics_by_clone.csv  
(required columns: Clone plus `{metric}_mean` and `{metric}_std` for every
metric listed below).

Metrics plotted
---------------
mu, IVCD, q_G, q_L, q_Gln, q_Glu, q_rP,
Y_XG, Y_XL, Y_XQln, Y_XQlu  
(extend the list in the code if new variables are added.)

Outputs
-------
PNG files in `outputs/figures_exp/`
(one file per metric, named `<metric>_bar.png`)

Author
------
Emiliano Balderas R. | 16 Jul 2025  
Last edit : 06 Aug 2025
"""


import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# ───── Configuration ─────────────────────────────────────────────────── #
CSV_PATH   = Path("outputs/kinetics_by_clone.csv")
FIGURE_DIR = Path("outputs/figures_exp")
FIGURE_DIR.mkdir(parents=True, exist_ok=True)

FIGSIZE, DPI = (8, 6), 300
AXES_RECT    = [0.15, 0.15, 0.78, 0.78]

sns.set_style("whitegrid")

# ───── Load data ─────────────────────────────────────────────────────── #
if not CSV_PATH.exists():
    raise FileNotFoundError(
        f"❌ File not found:\n  {CSV_PATH}\n"
        "Run `exp_phase_kinetics.py` first."
    )
df = pd.read_csv(CSV_PATH)

# ───── Palette ───────────────────────────────────────────────────────── #
clones = df["Clone"].tolist()
CLONE_COLOR = dict(zip(clones, sns.color_palette("tab10", len(clones))))

# ───── Plot helper ───────────────────────────────────────────────────── #
def plot_bar(metric: str, ylabel: str):
    mean_col, sd_col = f"{metric}_mean", f"{metric}_std"
    means, stds = df[mean_col], df[sd_col]

    fig = plt.figure(figsize=FIGSIZE, dpi=DPI)
    ax  = fig.add_axes(AXES_RECT)

    ax.bar(
        clones, means, yerr=stds,
        capsize=6, edgecolor="black", linewidth=0.7,
        color=[CLONE_COLOR[c] for c in clones],
        error_kw=dict(ecolor="black", linewidth=1.2)
    )

    ax.set_xlabel("Clone", fontsize=12)
    ax.set_ylabel(ylabel, fontsize=12)
    ax.set_title(metric, fontsize=13)
    ax.grid(axis="y", linestyle="--", alpha=0.5)
    fig.savefig(FIGURE_DIR / f"{metric}_bar.png")
    plt.close(fig)

# ───── Metrics list ──────────────────────────────────────────────────── #
METRICS = [
    ("mu",      r"μ (h$^{-1}$)"),
    ("IVCD",    r"IVCD (cell·h)"),
    ("q_G",     r"q$_{Glc}$ (pmol·cell$^{-1}$·h$^{-1}$)"),     # MOD
    ("q_L",     r"q$_{Lac}$ (pmol·cell$^{-1}$·h$^{-1}$)"),     # MOD
    ("q_Gln",   r"q$_{Gln}$ (pmol·cell$^{-1}$·h$^{-1}$)"),     # NEW
    ("q_Glu",   r"q$_{Glu}$ (pmol·cell$^{-1}$·h$^{-1}$)"),     # NEW
    ("q_rP",    r"q$_{rP}$ (pg·cell$^{-1}$·h$^{-1}$)"),        # NEW
    ("Y_XG",    r"Y$_{X/G}$ (cells·mol$^{-1}$)"),
    ("Y_XL",    r"Y$_{X/L}$ (cells·mol$^{-1}$)"),
    ("Y_XQln",  r"Y$_{X/Gln}$ (cells·mol$^{-1}$)"),            # NEW
    ("Y_XQlu",  r"Y$_{X/Glu}$ (cells·mol$^{-1}$)"),            # NEW
]

for metric, label in METRICS:
    if f"{metric}_mean" in df.columns:
        plot_bar(metric, label)

print("✓ Bar plots saved to ./outputs/figures_exp/")


✓ Bar plots saved to ./outputs/figures_exp/
